This notebook contains a few examples for bcc Fe to illustrate how VASP_job works

- Note: They only show how to create the inputs (set_calculation()), to submit the job, use:
   
   self.run_vasp()

In [1]:
import os
from subprocess import run
import numpy as np
from VASP_job.code.main import VASP_job

import pandas as pd
import ase
from ase.spacegroup import crystal

# Define a custom formatting function for vectors in pandas
format_vector = lambda vector: '[' + ', '.join(['{:.10f}'.format(x) for x in vector]) + ']'

In [2]:
dir_0 = os.getcwd()

In [3]:
# set executable
executable_path= "/home/emendive/workbench/work/from_claix/codes/VASP/vasp.5.4.4-flag4/bin"
potential_path = "/home/emendive/workbench/work/from_claix/codes/VASP/potentials/potpaw_PBE"

# simple FM bcc Fe

set working directory

In [4]:
cwd = dir_0 + "/FM"
if not os.path.exists(cwd):
   run("mkdir " + cwd, shell=True)

set crystal structure

In [5]:
a = 2.79 # A
alpha = 90
structure = crystal(('Fe'),
                       basis=[(0.00, 0.00, 0.00)],
                       spacegroup=229,
                       cellpar=[a, a, a, alpha, alpha, alpha])

set calculation

In [6]:
# create vasp class
vasp = VASP_job(verbose="low", executable_path=executable_path, potential_path=potential_path)

# update current working directory
vasp.io.cwd = cwd

In [7]:
# set magdirs for a FM state
number_of_atoms = len(structure)
vasp.io.magnetic_inputs.magdirs   = []
for i in range(number_of_atoms):
   if 'Fe' == structure.get_chemical_symbols()[i]:
      vasp.io.magnetic_inputs.magdirs.append([0, 0, 1])

In [8]:
# set calculation, which creates all input files
vasp.set_calculation(structure, vasp.io.magnetic_inputs)

visualize some inputs

In [9]:
vasp.df

,elements,positions,magdirs,ms,betahs,magmoms,B_CONSTRs
0,Fe,"[0.0, 0.0, 0.0]","[0, 0, 1]",False,inf,"[0, 0, 1]","[0, 0, 0]"
1,Fe,"[1.395, 1.395, 1.395]","[0, 0, 1]",False,inf,"[0, 0, 1]","[0, 0, 0]"


In [10]:
df_magmoms = vasp.df["magmoms"].apply(format_vector)
df_magmoms

0    [0.0000000000, 0.0000000000, 1.0000000000]
1    [0.0000000000, 0.0000000000, 1.0000000000]
Name: magmoms, dtype: object

In [11]:
print("kpoints =",vasp.structure.kpoints)
print("\nlattice vectors =\n",vasp.structure.lattice_vectors)
print("\nspecies =",vasp.structure.species)
print("\nelements =",vasp.structure.elements)

kpoints = 12 12 12

lattice vectors =
 [[2.79 0.   0.  ]
 [0.   2.79 0.  ]
 [0.   0.   2.79]]

species = ['Fe']

elements = ['Fe', 'Fe']


# FM bcc Fe, using a primitive cell, and relax structure

set working directory

In [12]:
cwd = dir_0 + "/FM_primitive_cell_relax"
if not os.path.exists(cwd):
   run("mkdir " + cwd, shell=True)

set crystal structure

In [13]:
a = 2.79 # A
alpha = 90
structure = crystal(('Fe'),
                       basis=[(0.00, 0.00, 0.00)],
                       spacegroup=229,
                       primitive_cell=True,
                       cellpar=[a, a, a, alpha, alpha, alpha])

set calculation

In [14]:
# create vasp class
vasp = VASP_job(verbose="low", executable_path=executable_path, potential_path=potential_path)

# update current working directory
vasp.io.cwd = cwd

In [15]:
# Set magdirs for a FM state
number_of_atoms = len(structure)
vasp.io.magnetic_inputs.magdirs   = []
for i in range(number_of_atoms):
   if 'Fe' == structure.get_chemical_symbols()[i]:
      vasp.io.magnetic_inputs.magdirs.append([0, 0, 1])

In [16]:
# first prepare relaxation
vasp.prepare_relaxation(ISIF="3")
# set calculation, which creates all input files
vasp.set_calculation(structure, vasp.io.magnetic_inputs)

visualize some inputs

In [17]:
vasp.df

,elements,positions,magdirs,ms,betahs,magmoms,B_CONSTRs
0,Fe,"[0.0, 0.0, 0.0]","[0, 0, 1]",False,inf,"[0, 0, 1]","[0, 0, 0]"


In [18]:
df_magmoms = vasp.df["magmoms"].apply(format_vector)
df_magmoms

0    [0.0000000000, 0.0000000000, 1.0000000000]
Name: magmoms, dtype: object

In [19]:
print("kpoints =",vasp.structure.kpoints)
print("\nlattice vectors =\n",vasp.structure.lattice_vectors)
print("\nspecies =",vasp.structure.species)
print("\nelements =",vasp.structure.elements)

kpoints = 12 12 12

lattice vectors =
 [[-1.395  1.395  1.395]
 [ 1.395 -1.395  1.395]
 [ 1.395  1.395 -1.395]]

species = ['Fe']

elements = ['Fe']


# AFM bcc Fe and change kpoints

set working directory

In [20]:
cwd = dir_0 + "/AFM_kpoints10x10x10"
if not os.path.exists(cwd):
   run("mkdir " + cwd, shell=True)

set crystal structure

In [21]:
a = 2.79 # A
alpha = 90
structure = crystal(('Fe'),
                       basis=[(0.00, 0.00, 0.00)],
                       spacegroup=229,
                       cellpar=[a, a, a, alpha, alpha, alpha])

set calculation

In [22]:
# create vasp class
vasp = VASP_job(verbose="low", executable_path=executable_path, potential_path=potential_path)

# update current working directory
vasp.io.cwd = cwd

# update kpoints
vasp.structure.kpoints = "10 10 10"

In [23]:
# Set magdirs for a AFM state
vasp.io.magnetic_inputs.magdirs   = []
vasp.io.magnetic_inputs.magdirs.append([0, 0, 1])
vasp.io.magnetic_inputs.magdirs.append([0, 0, -1])

In [24]:
# first prepare relaxation
vasp.prepare_relaxation(ISIF="3")
# set calculation, which creates all input files
vasp.set_calculation(structure, vasp.io.magnetic_inputs)

visualize some inputs

In [25]:
vasp.df

,elements,positions,magdirs,ms,betahs,magmoms,B_CONSTRs
0,Fe,"[0.0, 0.0, 0.0]","[0, 0, 1]",False,inf,"[0, 0, 1]","[0, 0, 0]"
1,Fe,"[1.395, 1.395, 1.395]","[0, 0, -1]",False,inf,"[0, 0, -1]","[0, 0, 0]"


In [26]:
df_magmoms = vasp.df["magmoms"].apply(format_vector)
df_magmoms

0     [0.0000000000, 0.0000000000, 1.0000000000]
1    [0.0000000000, 0.0000000000, -1.0000000000]
Name: magmoms, dtype: object

In [27]:
print("kpoints =",vasp.structure.kpoints)
print("\nlattice vectors =\n",vasp.structure.lattice_vectors)
print("\nspecies =",vasp.structure.species)
print("\nelements =",vasp.structure.elements)

kpoints = 10 10 10

lattice vectors =
 [[2.79 0.   0.  ]
 [0.   2.79 0.  ]
 [0.   0.   2.79]]

species = ['Fe']

elements = ['Fe', 'Fe']


# DLM bcc Fe

set working directory

In [28]:
cwd = dir_0 + "/DLM"
if not os.path.exists(cwd):
   run("mkdir " + cwd, shell=True)

set crystal structure

In [29]:
a = 2.79 # A
alpha = 90
structure = crystal(('Fe'),
                       basis=[(0.00, 0.00, 0.00)],
                       spacegroup=229,
                       cellpar=[a, a, a, alpha, alpha, alpha])

set calculation

In [30]:
# create vasp class
vasp = VASP_job(verbose="low", executable_path=executable_path, potential_path=potential_path)

# update current working directory
vasp.io.cwd = cwd

In [31]:
value_of_m = 0.0

number_of_atoms = len(structure)
vasp.io.magnetic_inputs.magdirs   = []
vasp.io.magnetic_inputs.ms    = []
for i in range(number_of_atoms):
   if 'Fe' == structure.get_chemical_symbols()[i]:
      vasp.io.magnetic_inputs.magdirs.append([0, 0, 1])
      vasp.io.magnetic_inputs.ms.append(value_of_m)

In [32]:
# first prepare bfields (constraints)
vasp.prepare_bfields(I_CONSTRAINED="4", LAMBDA="1")
# set calculation, which creates all input files
vasp.set_calculation(structure, vasp.io.magnetic_inputs)

visualize some inputs

In [33]:
vasp.df

,elements,positions,magdirs,ms,betahs,magmoms,B_CONSTRs
0,Fe,"[0.0, 0.0, 0.0]","[0, 0, 1]",0.0,0.0,"[0.7755158391951745, 0.34927788569939183, 0.52...","[0, 0, 0]"
1,Fe,"[1.395, 1.395, 1.395]","[0, 0, 1]",0.0,0.0,"[-0.3947049160123234, -0.7485935462390422, -0....","[0, 0, 0]"


In [34]:
df_magmoms = vasp.df["magmoms"].apply(format_vector)
df_magmoms

0       [0.7755158392, 0.3492778857, 0.5259088721]
1    [-0.3947049160, -0.7485935462, -0.5327435892]
Name: magmoms, dtype: object

In [35]:
print("kpoints =",vasp.structure.kpoints)
print("\nlattice vectors =\n",vasp.structure.lattice_vectors)
print("\nspecies =",vasp.structure.species)
print("\nelements =",vasp.structure.elements)

kpoints = 12 12 12

lattice vectors =
 [[2.79 0.   0.  ]
 [0.   2.79 0.  ]
 [0.   0.   2.79]]

species = ['Fe']

elements = ['Fe', 'Fe']


# Show default values

In [36]:
# create vasp class
vasp = VASP_job(verbose="high", executable_path=executable_path, potential_path=potential_path)


Your executable is:
   /home/emendive/workbench/work/from_claix/codes/VASP/vasp.5.4.4-flag4/bin/vasp_ncl

Your current working directory (cwd) is:
   /home/emendive/workbench/devel/gitclones/VASP_job/notebooks/examples/bccFe_FM/

Your bfields and relaxation flags are:
   bfields=False
   relaxation=False

Your default INCAR parameters are:
   SYSTEM = job
   PREC = Accurate
   ALGO = Fast
   ISTART = 0
   ICHARG = 2
   LREAL = .FALSE.
   LWAVE = .FALSE.
   LCHARG = .TRUE.
   LORBIT = 11
   ISMEAR = 1
   SIGMA = 0.005
   ISPIN = 2
   ENCUT = 500
   EDIFF = 1e-6
   NELM = 200
   LMAXMIX = 6
   LNONCOLLINEAR = .TRUE.
   NPAR = 5
   ISYM = 2

Your default INCAR parameters for constraining fields are:
   I_CONSTRAINED = 4
   LAMBDA = 1
   B_MIX = 1.0
   B_ref = 0.02
   N_MIX = 4
   E_PENALTY_MAX = 3.8
   LAMBDA_FIELD_MAX = 1e-3

Your default INCAR parameters for relaxation are:
   IBRION = 2
   ISIF = 3
   NSW = 100
   EDIFFG = -0.01

Your default RWIGS parameters are:
   N = 0.979
   Mn =